In [253]:
import json
import ujson
import glob
import time
import multiprocessing
import re
import os
import time
from gensim.models import word2vec, Word2Vec
from gensim.models.word2vec import PathLineSentences
from itertools import repeat
from scipy import spatial
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import codecs
from nltk.corpus import stopwords
import math
import functools
import operator

stopwords = set(stopwords.words('english'))

custom_stopwords = [s.split("'")[0] for s in stopwords]
custom_stopwords = [s for s in custom_stopwords if len(s) > 1]
print(custom_stopwords)

['mightn', 'again', 'some', 'very', 'ours', 'having', 'over', 'where', 'those', 'my', 'doing', 'her', 'as', 'with', 'is', 'both', 'you', 'against', 'nor', 'been', 'isn', 'ma', 'should', 'needn', 'am', 'before', 'more', 'than', 'shouldn', 'its', 'whom', 'this', 'down', 'here', 'their', 'she', 'out', 'yourselves', 'you', 'about', 'below', 'of', 'you', 'them', 'now', 'has', 'but', 'that', 'so', 'needn', 'myself', 'above', 'off', 'don', 'few', 'weren', 'and', 'your', 'him', 'then', 'each', 'what', 'between', 'can', 'they', 'will', 'we', 'our', 'it', 'these', 'any', 'ourselves', 'should', 'through', 'himself', 'she', 'wasn', 'haven', 'didn', 'were', 'the', 'by', 'mustn', 'hasn', 'under', 'wasn', 'he', 'me', 'yours', 'who', 'why', 'during', 'mustn', 'herself', 'if', 'being', 'didn', 'won', 'or', 'shouldn', 'shan', 'up', 'wouldn', 'hers', 'no', 'there', 'when', 've', 'couldn', 'for', 'it', 'after', 'to', 'most', 'his', 'hasn', 'aren', 'had', 'while', 'until', 'itself', 'other', 'll', 'ain', '

In [2]:
# load the extracted java-related GitHub data

def load_github_issues():
    with open('./output/issue-comments-revised.jsonl') as issue_comments_f:
        issue_list = []
        comments_list = []

        for line in issue_comments_f:
            obj = ujson.loads(line)

            comments_list.append({
                'body': obj['body'],
                'repo_name': obj['repo_name'],
                'html_url': obj['html_url'],
                'issue_id': obj['issue']['id']
            })

            issue = obj['issue']
            issue['repo_name'] = obj['repo_name']
            issue_list.append(issue)

        issues_df = pd.DataFrame(issue_list)
        issues_df = issues_df.drop_duplicates(subset=['id'])
        comments_df = pd.DataFrame(comments_list)
        
        return (issues_df, comments_df)


issues_df, comments_df = load_github_issues()
print("Number of issues: {}".format(len(issues_df.index)))
print("Number of comments: {}".format(len(comments_df.index)))

Number of issues: 627450
Number of comments: 1855870


In [3]:
# find the issues that contain code blocks
code_issues_df = issues_df.dropna(subset=['body'])
code_issues_df = code_issues_df[code_issues_df['body'].str.contains('```')]
print("Number of issues with code block/s: {}".format(len(code_issues_df.index)))

Number of issues with code block/s: 85318


In [4]:
comments_df.head()

,body,html_url,issue_id,repo_name
0,I can reproduce this with the below test snipp...,https://github.com/elastic/elasticsearch/issue...,264716524,elastic/elasticsearch
1,I don't really like the snackbars; I think we ...,https://github.com/ritvikkar/MovieFinder/issue...,285146015,ritvikkar/MovieFinder
2,Just an example to see how to use the `thymel...,https://github.com/jooby-project/jooby/issues/...,284439800,jooby-project/jooby
3,@nedtwigg Would it be practical for you to upg...,https://github.com/diffplug/spotless/issues/18...,285279535,diffplug/spotless
4,fixed,https://github.com/Alex-the-666/Ice_and_Fire/i...,258569936,Alex-the-666/Ice_and_Fire


In [6]:
def preprocess(doc):
    # remove all code blocks
    doc = re.sub(r'```([^```]*)```', '', doc)
            
    # remove urls
    doc = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)',             
        '', doc)
    
    # remove line break characters
    doc = re.sub(r'[\r\n]', ' ', doc)
    
    # remove apostrophes/suffixes
    doc = re.sub(r"'\w |\w' ", ' ', doc)
    
    # remove numbers
    doc = re.sub(r'(\d\.?)+', ' ', doc)
        
    # replace all punctuation except for full stop with space
    doc = re.sub(r'[^A-Za-z\.]', ' ', doc)
        
    # normalise full stops
    doc = re.sub(r'\s\.\.+', '.', doc)
    
    # remove more than 1 whitespace
    doc = re.sub('\s\s+', ' ', doc)
    
    # remove leading and trailing whitespace
    doc = doc.strip()
    
    return doc

def sent_tokenize(doc):
    docs = doc.split('. ')
    docs = [t for t in docs if t != '']
    return docs

def doc_tokenize(doc):
    doc = re.sub('\.', ' ', doc) # remove full stops
    
    tokens = [t.lower() for t in doc.split() if len(t) > 1]
    tokens = [t for t in tokens if t not in custom_stopwords]
    return tokens

def tokenize(sentence):
    sentence = re.sub('\.', ' ', sentence) # remove full stops
    
    tokens = sentence.split()
    tokens = [t.lower() for t in tokens if len(t) > 1]
    tokens = [t for t in tokens if t not in custom_stopwords]
    return tokens

In [9]:
# load the non-dreprecated java doc caveat sentences

def load_caveats():
    caveat_files_dir = './output/java_12_spec_caveat_sentences_revised/'
    caveats_list = []

    files = glob.glob(caveat_files_dir + '*.json')
    for file in files:
        with open(file) as f:
            arr = ujson.load(f)
            full_class_name = os.path.splitext(os.path.basename(file))[0]
            simple_class_name = full_class_name.split('.')[-1]
            for caveat in arr:
                if not caveat['deprecated'] and 'name' in caveat:
                    for sentence in caveat['sentences']:
                        caveats_list.append({
                            'simple_class_name': simple_class_name,
                            'full_class_name': full_class_name,
                            'api': caveat['name'],
                            'sentence': sentence,
                            'type': 'body'
                        })

                    # add all misc level sentences
                    for misc_obj in caveat['caveat_misc']:
                        if misc_obj['name'] in ['Parameters:', 'Throws:']:
                            for obj in misc_obj['list']:
                                for misc_sentence in obj['sentences']:
                                    caveats_list.append({
                                        'simple_class_name': simple_class_name,
                                        'full_class_name': full_class_name,
                                        'api': caveat['name'],
                                        'sentence': misc_sentence,
                                        'type': 'misc'
                                    })

                        else:
                            for s in misc_obj['list']:
                                caveats_list.append({
                                        'simple_class_name': simple_class_name,
                                        'full_class_name': full_class_name,
                                        'api': caveat['name'],
                                        'sentence': s,
                                        'type': 'misc'
                                    })

    return pd.DataFrame(caveats_list)

caveats = load_caveats()
print('Number of caveat sentences: {}'.format(len(caveats.index)))

Number of caveat sentences: 73831


In [10]:
caveats.head()

,api,full_class_name,sentence,simple_class_name,type
0,getBlockedTime,java.lang.management.ThreadInfo,This method returns -1 if thread contention mo...,ThreadInfo,body
1,getBlockedTime,java.lang.management.ThreadInfo,This statistic is reset when the thread conten...,ThreadInfo,body
2,getBlockedTime,java.lang.management.ThreadInfo,the approximate accumulated elapsed time in mi...,ThreadInfo,misc
3,getBlockedTime,java.lang.management.ThreadInfo,if the Java virtual machine does not support t...,ThreadInfo,misc
4,getWaitedTime,java.lang.management.ThreadInfo,This method returns -1 if thread contention mo...,ThreadInfo,body


In [11]:
caveats['preprocessed'] = caveats['sentence'].apply(lambda x: preprocess(x))
caveats['tokens'] = caveats['preprocessed'].map(lambda x: tokenize(x))

In [8]:
# Concatenate text in the issue all comments for each issue
# issue_to_concat_text = {}
# for index, row in code_issues_df.iterrows():
#     assoc_comments_df = comments_df[comments_df['issue_id'] == row['id']]
    
#     concat_text = row['title'] + row['body']
    
#     for j, comment_row in assoc_comments_df.iterrows():
#         concat_text += row['body']
        
#     issue_to_concat_text[row['id']] = concat_text

In [14]:
# Identify the relevant issues for all caveats
# apis = set()
# for caveat in caveats_list:
#     apis.add((caveat['simple_class_name'], caveat['api']))
    
# print('Number of apis: {}'.format(len(apis)))
    
# def get_relevant_issues(api_tuple, issue_to_text_dict):
#     relevant = []
#     for id in issue_to_text_dict:
#         if api_tuple[0] in issue_to_text_dict[id]:
#             if (api_tuple[0] == api_tuple[1]) or api_tuple[1] in issue_to_text_dict[id]:
#                 relevant.append(id)
            
#     return relevant

# p = multiprocessing.Pool(2)
# relevant_issues = p.starmap(get_relevant_issues, zip(get_relevant_issues, repeat(issue_to_concat_text)))

Number of apis: 21942


Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/thien/anaconda2/envs/nlp/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlo

In [17]:
# with open('./output/relevant_issues_for_caveats.jsonl', 'w+') as f:
#     for i, api in enumerate(apis):
#         f.write(ujson.dumps({'class': api[0], 'api': api[1], 'issue_ids': relevant_issues[i]}) + '\n')

In [12]:
def tokenize_sentence_list(sent_list):
    return [tokenize(x) for x in sent_list]

def calculate_preprocessed_comment_sentences(df):
    df['preprocessed_comments'] = df['body'].map(lambda x: preprocess(x))
    print('Preprocessed all comments...')
    
    df['tokenised_para'] = df['preprocessed_comments'].map(lambda x: doc_tokenize(x))
    print('Tokenized all paragraphs...')
    
    df['sentences'] = df['preprocessed_comments'].map(lambda x: sent_tokenize(x))
    print('Completed sentence tokenization...')
    
    df['tokenised_sentences'] = df['sentences'].map(lambda x: tokenize_sentence_list(x))
    print('Tokenized all sentences...')
                            
calculate_preprocessed_comment_sentences(comments_df)

Preprocessed all comments...
Tokenized all paragraphs...
Completed sentence tokenization...
Tokenized all sentences...


In [110]:
# Get all comments that contain non-empty preprocessed sentences
preprocessed_comments_df = comments_df[comments_df.astype(str)['tokenised_sentences'] != '[]']
print(len(comments_df.index))
print(len(preprocessed_comments_df.index))

1855870
1807851


In [378]:
# write the comment sentences to file, alongside relevant info to retrieve df row later
with open('./output/issue-comment-sentences-3.txt', 'w+') as f_out_sents, \
    open('./output/associated-sentence-df-index.txt', 'w+') as f_out_index, \
    open('./output/associated-sentence-issue-ids-3.txt', 'w+') as f_out_id:
        
    sent_str = ''
    index_str = ''
    id_str = ''
    
    c = 0
    for i in comments_df.index:
        issue_id = comments_df.loc[i, 'issue_id']
        sentences = comments_df.loc[i, 'tokenised_sentences']
        for sentence in sentences:
            sent_str += ' '.join(sentence) + '\n'
            index_str += str(i) + '\n'
            id_str += str(issue_id) + '\n'
            c += 1
            
            if c >= 100000:
                f_out_sents.write(sent_str)
                f_out_index.write(index_str)
                f_out_id.write(id_str)
                sent_str = ''
                index_str = ''
                id_str = ''
                
                c = 0
    
    if len(sent_str) > 0:
        f_out_sents.write(sent_str)
    if len(index_str) > 0:
        f_out_index.write(index_str)
    if len(id_str) > 0:
        f_out_id.write(id_str)    

In [63]:
cores = multiprocessing.cpu_count()

start = time.clock()
model = word2vec.Word2Vec(PathLineSentences('./output/issue-comment-sentences-2.txt'), size=100, window=5, min_count=5, workers=cores-1, iter=1, sg=1)
end =time.clock()
training_time=end-start
print('end training and cost ' + str(training_time)+ ' s')

end training and cost 450.77793999999994 s


In [64]:
# save word2vec model
model.save('./output/word2vec.model')
model.wv.save_word2vec_format('./output/word2vec.txt')

In [13]:
# calculate idf
idf = {}
with open('./output/issue-comment-sentences-2.txt') as f:
    start = time.clock()
    lines = f.readlines()
    print('Finished reading sentences from file...')    
    vocab = list(Word2Vec.load('./output/word2vec.model').wv.vocab.keys())
    N = len(lines)
    docs = [sentence.split() for sentence in lines]
    
    for doc in docs:
        for word in set(doc):
            if word not in idf:
                idf[word] = 1
            else:
                idf[word] += 1
    
    for word in idf:
        idf[word] = math.log(N / float(idf[word] + 1))
    
    end = time.clock()
    training_time=end-start
    print('IDF computation time: {}s'.format(training_time))

Finished reading sentences from file...
IDF computation time: 25.547523000000012s


In [186]:
s_avg = 0 # avg doc length
with open('./output/issue-comment-sentences-2.txt','r', encoding='utf-8') as f:
    lines = f.readlines()
    
    doc_lengths = [len(line.split()) for line in lines]
    s_avg = sum(doc_lengths) / len(doc_lengths)
    print("average document length: {}".format(s_avg)) 

# Calculate combination scores of word2vec and bm25
def bm25(doc, s2, idf):
    score = 0
    k1 = 1.5
    b = 0.75

    for w in doc:
        idf_s = idf.get(w, 1)
        bm25_ra = s2.count(w) * (k1 + 1)
        bm25_rb = s2.count(w) + k1 * (1 - b + b * len(s2) / s_avg)
        score += idf_s * (bm25_ra / bm25_rb)
    return score

def compute(s1, s2, voc):   
    v2 = np.array([voc[s] for s in s2 if s in voc])
    v2 = v2.sum(axis=0)

    v1 = np.array([voc[s] for s in s1 if s in voc])
    v1 = v1.sum(axis=0)
    
    return 1 - spatial.distance.cosine(v1, v2)

def cosine(sentences, s2, voc):
    s1_df_score = pd.Series(sentences)
    s1_df_score = s1_df_score.map(lambda x: compute(x, s2, voc))
    
    s1_df_score.dropna(inplace=True)
    return s1_df_score.sort_values(ascending=False).head(1)
        
def load_voc(file_voc):
    vector_file = codecs.open(file_voc, 'r', encoding='utf-8')
    line = vector_file.readline()
    voc_size, vec_dim = map(int, line.split(' '))
    embedding = dict()
    line = vector_file.readline()
    while line:
        items = line.split(' ')
        item = items[0]
        vec = np.array(items[1:], dtype='float32')
        embedding[item] = vec
        line = vector_file.readline()
    return embedding

average document length: 11.103317980494701


In [169]:
vocab = load_voc('./output/word2vec.txt')
    
relevant_issues = {}
with open('./output/relevant_issues_for_caveats.jsonl') as f:
    for line in f:
        obj = ujson.loads(line)
        relevant_issues[(obj['class'], obj['api'])] = obj['issue_ids']
print('Number of APIs with at least 1 relevant issue: {}'.format(len(relevant_issues)))

Number of APIs with at least 1 relevant issue: 21942


In [368]:
# TF-IDF score computation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_tf_idf_query_similarity(vectorizer, vecs, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    query_tfidf = vectorizer.transform([query])
    sim_scores = cosine_similarity(query_tfidf, vecs).flatten()
    return sim_scores

In [289]:
tf_idf_vectorizer = None
tf_idf_vectors = None
with open('./output/issue-comment-sentences-2.txt') as f:
    docs = f.readlines()
    
    tf_idf_vectorizer = TfidfVectorizer(lowercase=None)
    tf_idf_vectors = tf_idf_vectorizer.fit_transform(docs)
    print("Computed TF-IDF vectors for all documents...")

Computed TF-IDF vectors for all documents...


In [379]:
# create dicts to map issue sentences to metadata
issue_id_to_sentence_index = {}
with open('./output/associated-sentence-issue-ids-2.txt') as f2:
    ids = f2.readlines()
    ids = [int(x) for x in ids]
    
    for index, id in enumerate(ids):
        if not id in issue_id_to_sentence_index:
            issue_id_to_sentence_index[id] = []
        issue_id_to_sentence_index[id].append(index)
        
sentence_index_to_df_index = {}
with open('./output/associated-sentence-df-index.txt') as f:
    indices = f.readlines()
    indices = [int(x) for x in indices]

    for index, df_index in enumerate(indices):
        sentence_index_to_df_index[index] = df_index

In [381]:
print(1261355 in issue_id_to_sentence_index)

False


In [386]:
start = time.clock()

with open('./output/tfidf_results.jsonl', 'w+') as f:
    complete_tfidf_sim_results = ''
    c = 0
    # calculate scores for each caveat sentence
    for i in caveats.index:
        key = (caveats.loc[i,'simple_class_name'], caveats.loc[i,'api'])
        if key in relevant_issues:
            issue_ids = relevant_issues[key]

            if len(issue_ids) > 0:
                indices = []

                for id in issue_ids:
                    if id in issue_id_to_sentence_index:
                        indices += issue_id_to_sentence_index[id]

                if len(indices) > 0:
                    relevant_vecs = tf_idf_vectors[indices,:]
                    sim_scores = get_tf_idf_query_similarity(tf_idf_vectorizer, relevant_vecs, ' '.join(caveats.loc[i, 'tokens']))

                    scores = []
                    for j, score in enumerate(sim_scores):
                        if score > 0:
                            comment_index = sentence_index_to_df_index[indices[j]]
                            scores.append({
                                'score': score,
                                'sentence_index': indices[j]
                            })

                    complete_tfidf_sim_results += ujson.dumps({
                        'simple_class_name': key[0],
                        'api': key[1],
                        'caveat_sentence': caveats.loc[i, 'sentence'],
                        'caveat_sentence_id': i,
                        'tfidf_sim_scores': scores
                    }) + '\n'

                    c += 1

                    if c >= 2000:
                        f.write(complete_tfidf_sim_results)
                        
                        c = 0
                        complete_tfidf_sim_results = ''
                        
    if len(complete_tfidf_sim_results) > 0:
        f.write(complete_tfidf_sim_results)
           
    end = time.clock()
    training_time=end-start
    print('Cosine similarity for TF-IDF vectors computation time: {}s'.format(training_time))

Cosine similarity for TF-IDF vectors computation time: 617.8635890000005s


In [187]:
# # TESTING
# i = 111
# key = (caveats.loc[i,'simple_class_name'], caveats.loc[i,'api'])
# if key in relevant_issues:
#     issue_ids = relevant_issues[key]

#     if len(issue_ids) > 0:
#         caveat_sent = caveats.loc[i,'tokens']
#         combined_sim_results = []
#         bm25_results = []
#         word2vec_results = []

#         # retrieve issue comment sentences that are relevant
#         relevant_comments_df = preprocessed_comments_df[preprocessed_comments_df['issue_id'].isin(issue_ids)]

#         sim_w2v = relevant_comments_df['tokenised_sentences'].apply(cosine, s2=caveat_sent, voc=vocab)
#         sim_bm25 = relevant_comments_df['tokenised_para'].apply(bm25, s2=caveat_sent, idf=idf)

#         sim_bm25 = (sim_bm25 - sim_bm25.min()) / (sim_bm25.max() - sim_bm25.min())
#         sim_bm25 = pd.to_numeric(sim_bm25, downcast='float')
#         sim_w2v = pd.to_numeric(sim_w2v[0], downcast='float')

#         if len(sim_bm25) != 0 or len(sim_w2v) != 0:
#             # word2vec cosine similarity
#             for j in sim_w2v.index:
#                 if not np.isnan(sim_w2v[j]):
#                     word2vec_results.append({
#                         'score': str(sim_w2v[j]),
#                         'issue_number': int(relevant_comments_df.loc[j, 'issue_id']),
#                         'comment': relevant_comments_df.loc[j,'body']
#                     })

#             # bm25 score
#             for j in sim_bm25.index:
#                 if sim_bm25[j] != 0:
#                     bm25_results.append({
#                         'score': str(sim_bm25[j]),
#                         'issue_number': int(relevant_comments_df.loc[j, 'issue_id']),
#                         'comment': relevant_comments_df.loc[j,'body']
#                     })

#             # calculate combination similarity score
#             combined_sim = 0.5 * sim_bm25.add(0.5 * sim_w2v, fill_value=0)
#             for j in combined_sim.index:
#                 if combined_sim[j] > 0:
#                     combined_sim_results.append({
#                         'issue_number': int(relevant_comments_df.loc[j,'issue_id']),
#                         'comment': relevant_comments_df.loc[j,'body'],
#                         'score': str(combined_sim[j]),
#                     })

#             # Write results to relevant files
#             if len(word2vec_results) > 0:
#                 print(ujson.dumps({
#                     'simple_class_name': key[0],
#                     'api': key[1],
#                     'caveat_sentence': caveats.loc[i, 'sentence'],
#                     'caveat_sentence_id': i,
#                     'w2v_results': word2vec_results
#                 }) + '\n')

#             if len(bm25_results) > 0:
#                 print(ujson.dumps({
#                     'simple_class_name': key[0],
#                     'api': key[1],
#                     'caveat_sentence': caveats.loc[i, 'sentence'],
#                     'caveat_sentence_id': i,
#                     'bm25_results': bm25_results
#                 }) + '\n')

#             if len(combined_sim_results) > 0:
#                 print(ujson.dumps({
#                     'simple_class_name': key[0],
#                     'api': key[1],
#                     'caveat_sentence': caveats.loc[i, 'sentence'],
#                     'caveat_sentence_id': i,
#                     'combination_results': combined_sim_results
#                 }) + '\n')

{"simple_class_name":"InaccessibleObjectException","api":"InaccessibleObjectException","caveat_sentence":"Constructs an InaccessibleObjectException with no detail message.","caveat_sentence_id":111,"w2v_results":[{"score":"0.7709417","issue_number":272221304,"comment":"@jhaals proper fix would be appreciated if you have the time :)"},{"score":"0.69860953","issue_number":324006061,"comment":"Same here :(\r\nMy error log:\r\n```\r\nWARNING: An illegal reflective access operation has occurred\r\nWARNING: Illegal reflective access by org.codehaus.groovy.reflection.CachedClass$3$1 (file:\/Users\/macintosh\/Desktop\/jd-gui-osx-1.4.0\/JD-GUI.app\/Contents\/Resources\/Java\/jd-gui-1.4.0.jar) to method java.lang.Object.finalize()\r\nWARNING: Please consider reporting this to the maintainers of org.codehaus.groovy.reflection.CachedClass$3$1\r\nWARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations\r\nWARNING: All illegal access operations will be de

In [180]:
with open('./output/combined_sim_results_2.jsonl', 'w+') as f_combo_out, \
    open('./output/word2vec_results.jsonl', 'w+') as f_w2v_out, \
    open('./output/bm25_results.jsonl', 'w+') as f_bm25_out, \
    open('./output/ir_error_log.jsonl', 'w+') as f_err:
        
    for i in caveats.index:
        key = (caveats.loc[i,'simple_class_name'], caveats.loc[i,'api'])
        if key in relevant_issues:
            issue_ids = relevant_issues[key]

            if len(issue_ids) > 0:
                try:
                    caveat_sent = caveats.loc[i,'tokens']
                    combined_sim_results = []
                    bm25_results = []
                    word2vec_results = []

                    # retrieve issue comment sentences that are relevant
                    relevant_comments_df = preprocessed_comments_df[preprocessed_comments_df['issue_id'].isin(issue_ids)]

                    sim_w2v = relevant_comments_df['tokenised_sentences'].apply(cosine, s2=caveat_sent, voc=vocab)
                    sim_bm25 = relevant_comments_df['tokenised_para'].apply(bm25, s2=caveat_sent, idf=idf)

                    sim_bm25 = (sim_bm25 - sim_bm25.min()) / (sim_bm25.max() - sim_bm25.min())
                    sim_bm25 = pd.to_numeric(sim_bm25, downcast='float')
                    sim_w2v = pd.to_numeric(sim_w2v[0], downcast='float')

                    if len(sim_bm25) != 0 or len(sim_w2v) != 0:
                        # word2vec cosine similarity
                        for j in sim_w2v.index:
                            if not np.isnan(sim_w2v[j]):
                                word2vec_results.append({
                                    'score': str(sim_w2v[j]),
                                    'issue_number': int(relevant_comments_df.loc[j, 'issue_id']),
                                    'comment': relevant_comments_df.loc[j,'body']
                                })

                        # bm25 score
                        for j in sim_bm25.index:
                            if sim_bm25[j] != 0:
                                bm25_results.append({
                                    'score': str(sim_bm25[j]),
                                    'issue_number': int(relevant_comments_df.loc[j, 'issue_id']),
                                    'comment': relevant_comments_df.loc[j,'body']
                                })

                        # calculate combination similarity score
                        combined_sim = 0.5 * sim_bm25.add(0.5 * sim_w2v, fill_value=0)
                        for j in combined_sim.index:
                            if combined_sim[j] > 0:
                                combined_sim_results.append({
                                    'issue_number': int(relevant_comments_df.loc[j,'issue_id']),
                                    'comment': relevant_comments_df.loc[j,'body'],
                                    'score': str(combined_sim[j]),
                                })

                        # Write results to relevant files
                        if len(word2vec_results) > 0:
                            f_w2v_out.write(ujson.dumps({
                                'simple_class_name': key[0],
                                'api': key[1],
                                'caveat_sentence': caveats.loc[i, 'sentence'],
                                'caveat_sentence_id': i,
                                'w2v_results': word2vec_results
                            }) + '\n')

                        if len(bm25_results) > 0:
                            f_bm25_out.write(ujson.dumps({
                                'simple_class_name': key[0],
                                'api': key[1],
                                'caveat_sentence': caveats.loc[i, 'sentence'],
                                'caveat_sentence_id': i,
                                'bm25_results': bm25_results
                            }) + '\n')

                        if len(combined_sim_results) > 0:
                            f_combo_out.write(ujson.dumps({
                                'simple_class_name': key[0],
                                'api': key[1],
                                'caveat_sentence': caveats.loc[i, 'sentence'],
                                'caveat_sentence_id': i,
                                'combination_results': combined_sim_results
                            }) + '\n')
                except Exception as e:
                    f_err.write(ujson.dumps({'caveat_index': i, 'error': e}) + '\n')

IN RETURN
0    0.741139
dtype: float64
IN RETURN
0    0.841054
dtype: float64
IN RETURN
2    0.866139
dtype: float64
IN RETURN
0    0.8738
dtype: float64
IN RETURN
0    0.742
dtype: float64
IN RETURN
1    0.855557
dtype: float64
IN RETURN
0    0.790563
dtype: float64
IN RETURN
0    0.83637
dtype: float64
IN RETURN
0    0.867499
dtype: float64
IN RETURN
0    0.482622
dtype: float64
IN RETURN
3    0.877329
dtype: float64
IN RETURN
0    0.78274
dtype: float64
IN RETURN
0    0.560772
dtype: float64
IN RETURN
0    0.660109
dtype: float64
IN RETURN
0    0.866494
dtype: float64
IN RETURN
0    0.801281
dtype: float64
IN RETURN
0    0.828071
dtype: float64
IN RETURN
1    0.802363
dtype: float64
IN RETURN
0    0.827456
dtype: float64
IN RETURN
0    0.814058
dtype: float64
IN RETURN
3    0.906488
dtype: float64
IN RETURN
3    0.891877
dtype: float64
IN RETURN
0    0.783468
dtype: float64
IN RETURN
0    0.883098
dtype: float64
IN RETURN
0    0.811291
dtype: float64
IN RETURN
0    0.901607
dtype: f

dtype: float64
IN RETURN
1    0.638261
dtype: float64
IN RETURN
0    0.72923
dtype: float64
IN RETURN
0    0.685764
dtype: float64
IN RETURN
3    0.814636
dtype: float64
IN RETURN
3    0.788242
dtype: float64
IN RETURN
0    0.678641
dtype: float64
IN RETURN
1    0.856549
dtype: float64
IN RETURN
0    0.757953
dtype: float64
IN RETURN
0    0.816966
dtype: float64
IN RETURN
0    0.80984
dtype: float64
IN RETURN
0    0.510616
dtype: float64
IN RETURN
0    0.534862
dtype: float64
IN RETURN
0    0.469229
dtype: float64
IN RETURN
0    0.424768
dtype: float64
IN RETURN
0    0.544201
dtype: float64
IN RETURN
1    0.72404
dtype: float64
IN RETURN
0    0.57485
dtype: float64
IN RETURN
0    0.499421
dtype: float64
IN RETURN
0    0.560242
dtype: float64
IN RETURN
2    0.536615
dtype: float64
IN RETURN
1    0.683505
dtype: float64
IN RETURN
0    0.526929
dtype: float64
IN RETURN
0    0.68232
dtype: float64
IN RETURN
7    0.86176
dtype: float64
IN RETURN
0    0.76975
dtype: float64
IN RETURN
1    0.

IN RETURN
0    0.763996
dtype: float64
IN RETURN
1    0.663972
dtype: float64
IN RETURN
3    0.801685
dtype: float64
IN RETURN
0    0.851948
dtype: float64
IN RETURN
0    0.718965
dtype: float64
IN RETURN
0    0.595744
dtype: float64
IN RETURN
1    0.540388
dtype: float64
IN RETURN
0    0.815472
dtype: float64
IN RETURN
0    0.660223
dtype: float64
IN RETURN
0    0.739896
dtype: float64
IN RETURN
1    0.912432
dtype: float64
IN RETURN
0    0.772678
dtype: float64
IN RETURN
0    0.850837
dtype: float64
IN RETURN
0    0.644585
dtype: float64
IN RETURN
0    0.680962
dtype: float64
IN RETURN
1    0.766842
dtype: float64
IN RETURN
0    0.817603
dtype: float64
IN RETURN
0    0.711929
dtype: float64
IN RETURN
1    0.815491
dtype: float64
IN RETURN
0    0.816583
dtype: float64
IN RETURN
0    0.733568
dtype: float64
IN RETURN
0    0.666368
dtype: float64
IN RETURN
0    0.852082
dtype: float64
IN RETURN
0    0.707042
dtype: float64
IN RETURN
3    0.807683
dtype: float64
IN RETURN
2    0.851147
d

IN RETURN
1    0.581519
dtype: float64
IN RETURN
0    0.849723
dtype: float64
IN RETURN
0    0.710287
dtype: float64
IN RETURN
0    0.745428
dtype: float64
IN RETURN
1    0.919762
dtype: float64
IN RETURN
0    0.797073
dtype: float64
IN RETURN
0    0.877534
dtype: float64
IN RETURN
0    0.67086
dtype: float64
IN RETURN
0    0.720176
dtype: float64
IN RETURN
1    0.758704
dtype: float64
IN RETURN
0    0.838515
dtype: float64
IN RETURN
0    0.733706
dtype: float64
IN RETURN
1    0.843983
dtype: float64
IN RETURN
0    0.844774
dtype: float64
IN RETURN
0    0.769947
dtype: float64
IN RETURN
0    0.693098
dtype: float64
IN RETURN
0    0.859276
dtype: float64
IN RETURN
0    0.725965
dtype: float64
IN RETURN
3    0.836424
dtype: float64
IN RETURN
2    0.88659
dtype: float64
IN RETURN
1    0.680871
dtype: float64
IN RETURN
0    0.638076
dtype: float64
IN RETURN
0    0.89178
dtype: float64
IN RETURN
0    0.88543
dtype: float64
IN RETURN
0    0.785654
dtype: float64
IN RETURN
0    0.852544
dtype

dtype: float64
IN RETURN
1    0.838669
dtype: float64
IN RETURN
0    0.759185
dtype: float64
IN RETURN
0    0.640042
dtype: float64
IN RETURN
0    0.887783
dtype: float64
IN RETURN
0    0.832413
dtype: float64
IN RETURN
1    0.885311
dtype: float64
IN RETURN
0    0.858148
dtype: float64
IN RETURN
0    0.858721
dtype: float64
IN RETURN
2    0.81719
dtype: float64
IN RETURN
0    0.835607
dtype: float64
IN RETURN
1    0.823937
dtype: float64
IN RETURN
1    0.758894
dtype: float64
IN RETURN
0    0.762802
dtype: float64
IN RETURN
0    0.823945
dtype: float64
IN RETURN
1    0.891065
dtype: float64
IN RETURN
0    0.876658
dtype: float64
IN RETURN
0    0.861405
dtype: float64
IN RETURN
17    0.918434
dtype: float64
IN RETURN
1    0.812177
dtype: float64
IN RETURN
0    0.729688
dtype: float64
IN RETURN
2    0.857312
dtype: float64
IN RETURN
2    0.832743
dtype: float64
IN RETURN
0    0.616985
dtype: float64
IN RETURN
0    0.859345
dtype: float64
IN RETURN
0    0.683042
dtype: float64
IN RETURN


...

IN RETURN
1    0.740983
dtype: float64
IN RETURN
1    0.763341
dtype: float64
IN RETURN
1    0.566795
dtype: float64
IN RETURN
1    0.563353
dtype: float64
IN RETURN
0    0.62846
dtype: float64
IN RETURN
0    0.621855
dtype: float64
IN RETURN
1    0.689891
dtype: float64
IN RETURN
2    0.648412
dtype: float64
IN RETURN
1    0.775681
dtype: float64
IN RETURN
3    0.795918
dtype: float64
IN RETURN
4    0.650538
dtype: float64
IN RETURN
2    0.685473
dtype: float64
IN RETURN
6    0.768477
dtype: float64
IN RETURN
1    0.684098
dtype: float64
IN RETURN
2    0.654219
dtype: float64
IN RETURN
0    0.528749
dtype: float64
IN RETURN
0    0.420853
dtype: float64
IN RETURN
1    0.590157
dtype: float64
IN RETURN
0    0.62491
dtype: float64
IN RETURN
2    0.662197
dtype: float64
IN RETURN
0    0.601881
dtype: float64
IN RETURN
2    0.564189
dtype: float64
IN RETURN
2    0.623534
dtype: float64
IN RETURN
0    0.508834
dtype: float64
IN RETURN
5    0.844155
dtype: float64
IN RETURN
1    0.844682
dty

dtype: float64
IN RETURN
0    0.506729
dtype: float64
IN RETURN
0    0.454368
dtype: float64
IN RETURN
0    0.26304
dtype: float64
IN RETURN
0    0.510648
dtype: float64
IN RETURN
0    0.510375
dtype: float64
IN RETURN
5    0.575358
dtype: float64
IN RETURN
0    0.480071
dtype: float64
IN RETURN
0    0.274158
dtype: float64
IN RETURN
1    0.503377
dtype: float64
IN RETURN
0    0.499287
dtype: float64
IN RETURN
0    0.44014
dtype: float64
IN RETURN
1    0.496463
dtype: float64
IN RETURN
0    0.472968
dtype: float64
IN RETURN
0    0.392282
dtype: float64
IN RETURN
1    0.471893
dtype: float64
IN RETURN
18    0.56875
dtype: float64
IN RETURN
1    0.478225
dtype: float64
IN RETURN
1    0.497365
dtype: float64
IN RETURN
1    0.622439
dtype: float64
IN RETURN
0    0.685505
dtype: float64
IN RETURN
3    0.542862
dtype: float64
IN RETURN
0    0.424562
dtype: float64
IN RETURN
2    0.545392
dtype: float64
IN RETURN
0    0.50868
dtype: float64
IN RETURN
3    0.47958
dtype: float64
IN RETURN
1   

IN RETURN
1    0.76663
dtype: float64
IN RETURN
0    0.832739
dtype: float64
IN RETURN
1    0.77786
dtype: float64
IN RETURN
0    0.810201
dtype: float64
IN RETURN
0    0.701675
dtype: float64
IN RETURN
2    0.799861
dtype: float64
IN RETURN
0    0.789557
dtype: float64
IN RETURN
4    0.852287
dtype: float64
IN RETURN
0    0.814049
dtype: float64
IN RETURN
0    0.750414
dtype: float64
IN RETURN
0    0.800648
dtype: float64
IN RETURN
1    0.83616
dtype: float64
IN RETURN
3    0.798362
dtype: float64
IN RETURN
2    0.810897
dtype: float64
IN RETURN
0    0.798809
dtype: float64
IN RETURN
0    0.705924
dtype: float64
IN RETURN
1    0.816396
dtype: float64
IN RETURN
0    0.698375
dtype: float64
IN RETURN
0    0.657361
dtype: float64
IN RETURN
2    0.694429
dtype: float64
IN RETURN
0    0.350047
dtype: float64
IN RETURN
2    0.759079
dtype: float64
IN RETURN
0    0.760816
dtype: float64
IN RETURN
1    0.764887
dtype: float64
IN RETURN
0    0.757908
dtype: float64
IN RETURN
0    0.762236
dtyp

IN RETURN
0    0.572556
dtype: float64
IN RETURN
0    0.630533
dtype: float64
IN RETURN
1    0.530966
dtype: float64
IN RETURN
0    0.544353
dtype: float64
IN RETURN
1    0.548213
dtype: float64
IN RETURN
0    0.52327
dtype: float64
IN RETURN
0    0.530061
dtype: float64
IN RETURN
5    0.660305
dtype: float64
IN RETURN
0    0.546734
dtype: float64
IN RETURN
0    0.443729
dtype: float64
IN RETURN
0    0.482521
dtype: float64
IN RETURN
1    0.557441
dtype: float64
IN RETURN
4    0.520889
dtype: float64
IN RETURN
2    0.500772
dtype: float64
IN RETURN
0    0.522924
dtype: float64
IN RETURN
0    0.426541
dtype: float64
IN RETURN
2    0.569497
dtype: float64
IN RETURN
0    0.480105
dtype: float64
IN RETURN
1    0.435857
dtype: float64
IN RETURN
2    0.467893
dtype: float64
IN RETURN
0    0.224821
dtype: float64
IN RETURN
0    0.561469
dtype: float64
IN RETURN
0    0.526549
dtype: float64
IN RETURN
1    0.522151
dtype: float64
IN RETURN
0    0.49799
dtype: float64
IN RETURN
0    0.6441
dtype

KeyboardInterrupt: 